In [29]:
import os
import sys
import numpy as np
from datetime import datetime

import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import VOCDetection
import torchvision.transforms as transforms
from torchvision.models import alexnet, resnet50
from torchvision.transforms import functional as F
from xml.etree import ElementTree as ET
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# models from torchvision
import torchvision.models as models
from torchvision.ops import RoIPool

sys.path.append('../../')  
from Object_Detection.RCNN import selective_search
from torchmetrics import Accuracy
from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter


# Get Fast RCNN from torchvision


In [2]:
def parse_annotations(annotation):
    objects = annotation['annotation']['object']
    boxes = []
    classes = []
    for obj in objects:
        xmin = float(obj['bndbox']['xmin'])
        ymin = float(obj['bndbox']['ymin'])
        xmax = float(obj['bndbox']['xmax'])
        ymax = float(obj['bndbox']['ymax'])
        boxes.append([xmin, ymin, xmax, ymax])
        classes.append(obj['name'])
    return torch.tensor(boxes), classes  # Convert boxes to tensors, keep classes as list or map them to integers


In [3]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((800, 800)),  # Resize images to a common size
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomVOCDataset(VOCDetection):
    def __getitem__(self, index):
        img, target = super(CustomVOCDataset, self).__getitem__(index)
        boxes, classes = parse_annotations(target)
        return img, boxes, classes

# Update the dataset instance with the custom class
dataset = CustomVOCDataset(root='./data/VOCdevkit/VOC2012', year='2012', image_set='train', download=True, transform=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)


Using downloaded and verified file: ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar
Extracting ./data/VOCdevkit/VOC2012\VOCtrainval_11-May-2012.tar to ./data/VOCdevkit/VOC2012


In [34]:
gs = selective_search.get_selective_search()

counter, start_index = 0, 0

# Assuming 'voc_dataset' is iterable with (image, annot)
for image, boxes, classes in data_loader:
    
    counter += 1
    # Counter for images processed
    print(f'Processing image {counter}')

    print(image.shape)
    # Get proposals and annotations similar to your original code
    image_array = image.permute(0, 2, 3, 1).numpy()

    image_array = np.array(image_array)[0]
    print(image_array.shape)
    selective_search.config(gs, image_array, strategy='q')
    rects = selective_search.get_rects(gs)


    if counter == 1:
        break

Processing image 1
torch.Size([1, 3, 800, 800])
(800, 800, 3)


In [ ]:
# Load a pre-trained model
base_model = resnet50(pretrained=True)
# Remove the final classification layers
base_model = nn.Sequential(*list(base_model.children())[:-2])


In [7]:
# Intermediate RoI pooling layer that takes selected region proposals and extracts features
# RoI pooling
roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1.0/16)


In [8]:
num_classes = 21  # PASCAL VOC classes + background
classifier = torch.nn.Sequential(
    torch.nn.Linear(512 * 7 * 7, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, num_classes + 4)  # class scores and bbox regressor outputs
)



In [30]:
num_epochs = 10
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)

# Training loop
classifier.to(device)
base_model.to(device)

# Label encoder to convert string labels to integers
label_encoder = LabelEncoder()
label_encoder.fit(['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 
                   'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'background'])



LabelEncoder()

In [ ]:

# Define the backbone network
base_model = resnet50(pretrained=True)
base_model = nn.Sequential(*list(base_model.children())[:-2])

# Define the RoI Pooling and classification head
roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1.0 / 16)

num_classes = 21  # PASCAL VOC classes + background
classifier = nn.Sequential(
    nn.Linear(2048 * 7 * 7, 4096),  # Adjusted based on the RoI Pooling output
    nn.ReLU(inplace=True),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, num_classes + 4)  # class scores and bbox regressor outputs
)


# Label encoder to convert string labels to integers
label_encoder = LabelEncoder()
label_encoder.fit(['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 
                   'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor', 'background'])

# Hyperparameters
num_epochs = 10
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loss function and optimizer
criterion_cls = nn.CrossEntropyLoss()
criterion_reg = nn.SmoothL1Loss()
optimizer = optim.SGD(classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)

# Training loop
classifier.to(device)
base_model.to(device)

for epoch in range(num_epochs):
    classifier.train()
    base_model.train()

    running_loss = 0.0
    for i, (images, boxes, targets) in enumerate(data_loader):
        images = images.to(device)
        boxes = [box.to(device) for box in boxes]  # Assuming boxes is a list of tensors
        
        # Encode the string labels to integers
        label_targets = [torch.tensor(label_encoder.transform(target)).to(device) for target in targets]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        features = base_model(images)
        print("Feature map shape:", features.shape)  # Debugging feature map shape
        
        # Apply RoI pooling
        roi_pooled_features = roi_pool(features, boxes)
        print("RoI pooled feature shape:", roi_pooled_features.shape)  # Debugging RoI pooled feature shape
        
        # Flatten RoI pooled features
        roi_pooled_features = roi_pooled_features.view(roi_pooled_features.size(0), -1)
        print("Flattened RoI pooled feature shape:", roi_pooled_features.shape)  # Debugging flattened feature shape

        # Forward pass through the classifier
        outputs = classifier(roi_pooled_features)

        # Split outputs into class scores and bounding box regressions
        cls_scores = outputs[:, :num_classes]
        bbox_regressions = outputs[:, num_classes:]

        # Compute losses for each RoI
        cls_loss = 0
        reg_loss = 0
        if i == 1:
            print("Run the loop once")
            break